In [1]:
import pandas as pd
import numpy as np 
import consultas as cst

In [ ]:
#cantidad total de septiembre 

query=f'''
SELECT COUNT(*) AS TOTAL_MES FROM (
SELECT  *
FROM AGG_DL_PJ_QUALITY
WHERE PERIODO_PROCESO_CODIGO=202209
ORDER BY FECHA_PRED DESC )
'''

In [55]:
#Ccantidad total de consultor canal nombre
query=f'''
SELECT      A.CONSULTOR_CANAL_NOMBRE,
            COUNT(*) AS TOTAL_CONSULTOR_CANAL
FROM AGG_DL_PJ_QUALITY A
JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
WHERE A.PERIODO_PROCESO_CODIGO=202209
AND   B.PERIODO_PROCESO_CODIGO=202209
AND B.SERVICIO='Postpaid'
GROUP BY A.CONSULTOR_CANAL_NOMBRE
ORDER BY TOTAL_CONSULTOR_CANAL DESC

'''
consultor_canal_nombre=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [56]:
#generar el % de columna
suma=consultor_canal_nombre['TOTAL_CONSULTOR_CANAL'].sum()
consultor_canal_nombre['PORCENTAJE']=consultor_canal_nombre['TOTAL_CONSULTOR_CANAL']/suma *100

In [57]:
consultor_canal_nombre

,CONSULTOR_CANAL_NOMBRE,TOTAL_CONSULTOR_CANAL,PORCENTAJE
0,External organization,3651,87.198471
1,Tiendas,217,5.182708
2,Internal organization,179,4.275137
3,Cadenas,112,2.674946
4,Fuerza De Venta Indirecta,14,0.334368
5,Kioscos,5,0.119417
6,Self-employed,4,0.095534
7,Tiendas Express,2,0.047767
8,Outbound,1,0.023883
9,Inbound,1,0.023883


In [61]:
#QUERY QUE AGRUPA EL SUBCANAL
query=f'''
SELECT      A.CONSULTOR_SUBCANAL_NOMBRE,
            COUNT(*) AS TOTAL_CONSULTOR_SUBCANAL
FROM AGG_DL_PJ_QUALITY A
JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND A.CONSULTOR_CANAL_NOMBRE='Fuerza De Venta Indirecta'
AND B.PERIODO_PROCESO_CODIGO=202209
AND B.SERVICIO='Prepaid'
GROUP BY A.CONSULTOR_SUBCANAL_NOMBRE
ORDER BY TOTAL_CONSULTOR_SUBCANAL DESC


'''
consultor_subcanal_nombre=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [62]:
#hallar los % de los subcanales
suma=consultor_subcanal_nombre['TOTAL_CONSULTOR_SUBCANAL'].sum()
consultor_subcanal_nombre['PORCENTAJE']=consultor_subcanal_nombre['TOTAL_CONSULTOR_SUBCANAL']/suma *100

In [63]:
consultor_subcanal_nombre

,CONSULTOR_SUBCANAL_NOMBRE,TOTAL_CONSULTOR_SUBCANAL,PORCENTAJE
0,TAT,5531,68.742232
1,Tropas,2374,29.505344
2,Partner,122,1.516281
3,Tropas Directas,7,0.087000
4,Freelance,7,0.087000
5,Medios Impresos,2,0.024857
6,Masificadores,2,0.024857
7,None,1,0.012429


In [16]:
#query que agrupa por prediccion 
query=f'''
SELECT      CONSULTOR_SUBCANAL_NOMBRE,
            PREDICCION,
            COUNT(*) AS TOTAL_CONSULTOR_SUBCANAL_PRED
FROM AGG_DL_PJ_QUALITY
WHERE PERIODO_PROCESO_CODIGO=202210
AND CONSULTOR_CANAL_NOMBRE='Fuerza De Venta Indirecta'
GROUP BY CONSULTOR_SUBCANAL_NOMBRE,PREDICCION
ORDER BY CONSULTOR_SUBCANAL_NOMBRE,PREDICCION
'''
consultor_subcanal_nombre_pred=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
consultor_subcanal_general=consultor_subcanal_nombre_pred.merge(consultor_subcanal_nombre,left_on='CONSULTOR_SUBCANAL_NOMBRE',right_on='CONSULTOR_SUBCANAL_NOMBRE',how='left')

In [18]:
consultor_subcanal_general['PORCENTAJE_PRED']=consultor_subcanal_general['TOTAL_CONSULTOR_SUBCANAL_PRED']/consultor_subcanal_general['TOTAL_CONSULTOR_SUBCANAL']*100

In [19]:
consultor_subcanal_general=consultor_subcanal_general.sort_values('TOTAL_CONSULTOR_SUBCANAL',ascending=False)

In [20]:
consultor_subcanal_general

,CONSULTOR_SUBCANAL_NOMBRE,PREDICCION,TOTAL_CONSULTOR_SUBCANAL_PRED,TOTAL_CONSULTOR_SUBCANAL,PORCENTAJE,PORCENTAJE_PRED
12,TAT,0,36540,69330,55.245229,52.704457
13,TAT,1,32790,69330,55.245229,47.295543
19,Tropas,1,41749,52638,41.944301,79.313424
18,Tropas,0,10889,52638,41.944301,20.686576
10,Partner,0,519,2559,2.039125,20.281360
11,Partner,1,2040,2559,2.039125,79.718640
4,Freelance,0,398,480,0.382485,82.916667
5,Freelance,1,82,480,0.382485,17.083333
8,Medios Impresos,0,164,175,0.139448,93.714286
9,Medios Impresos,1,11,175,0.139448,6.285714


In [21]:
#query del total de cada tropa
subcanal='Tropas'
query=f'''
SELECT      CONSULTOR_NOMBRE_NIVEL_8,
            COUNT(*) AS TOTAL_CONSULTOR_NOMBRE_NIVEL_8
FROM AGG_DL_PJ_QUALITY
WHERE PERIODO_PROCESO_CODIGO=202210
AND CONSULTOR_CANAL_NOMBRE='Fuerza De Venta Indirecta'
AND CONSULTOR_SUBCANAL_NOMBRE='{subcanal}'
GROUP BY CONSULTOR_NOMBRE_NIVEL_8
ORDER BY CONSULTOR_NOMBRE_NIVEL_8
'''
dealer_tropas=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [22]:
suma=dealer_tropas['TOTAL_CONSULTOR_NOMBRE_NIVEL_8'].sum()
dealer_tropas['PORCENTAJE']=dealer_tropas['TOTAL_CONSULTOR_NOMBRE_NIVEL_8']/suma *100

In [23]:
dealer_tropas

,CONSULTOR_NOMBRE_NIVEL_8,TOTAL_CONSULTOR_NOMBRE_NIVEL_8,PORCENTAJE
0,12g Comunicaciones Sas,7,0.013298
1,Alba Jeanette Forero Osorio Eu,2,0.003800
2,Aloha Comunicaciones Sas,677,1.286143
3,Avatar Telcos s.a.s,48,0.091189
4,Beyotta Sas Beyotta Sas,19,0.036096
5,Bldistribuciones Sas,14,0.026597
6,Buendia Communication Digital S.A.S,28,0.053194
7,CORPORACION ACOMPANAMIENTO A EMPRESAS SIGLA C...,47,0.089289
8,Ccomunicaciones Nortec Sas,900,1.709791
9,Clan Movil,19,0.036096


In [24]:
#QUERY PARA SUBCANAL
subcanal='Tropas' 
query=f'''
SELECT      CONSULTOR_NOMBRE_NIVEL_8,
            PREDICCION,
            COUNT(*) AS TOTAL_CONSULTOR_NOMBRE_PRED
FROM AGG_DL_PJ_QUALITY
WHERE PERIODO_PROCESO_CODIGO=202210
AND CONSULTOR_CANAL_NOMBRE='Fuerza De Venta Indirecta'
AND CONSULTOR_SUBCANAL_NOMBRE='{subcanal}'
GROUP BY CONSULTOR_NOMBRE_NIVEL_8,PREDICCION
ORDER BY CONSULTOR_NOMBRE_NIVEL_8,PREDICCION
'''
dealer_tropas_pred=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [25]:
dealer_tropas_pred

,CONSULTOR_NOMBRE_NIVEL_8,PREDICCION,TOTAL_CONSULTOR_NOMBRE_PRED
0,12g Comunicaciones Sas,0,7
1,Alba Jeanette Forero Osorio Eu,0,2
2,Aloha Comunicaciones Sas,0,62
3,Aloha Comunicaciones Sas,1,615
4,Avatar Telcos s.a.s,0,21
...,...,...,...
82,Ventas 577 Ltda,1,2809
83,WAYIRA NET S.A.S,0,1
84,WAYIRA NET S.A.S,1,1
85,aliado profesional sas,0,95


In [26]:
dealer_tropas_general=dealer_tropas_pred.merge(dealer_tropas,left_on='CONSULTOR_NOMBRE_NIVEL_8',right_on='CONSULTOR_NOMBRE_NIVEL_8',how='left')

In [27]:
dealer_tropas_general['PORCENTAJE_PRED']=dealer_tropas_general['TOTAL_CONSULTOR_NOMBRE_PRED']/dealer_tropas_general['TOTAL_CONSULTOR_NOMBRE_NIVEL_8']*100

In [28]:
dealer_tropas_general=dealer_tropas_general.sort_values(['TOTAL_CONSULTOR_NOMBRE_NIVEL_8','PREDICCION'],ascending=False)

In [29]:
dealer_tropas_general[dealer_tropas_general['PREDICCION']==1]

,CONSULTOR_NOMBRE_NIVEL_8,PREDICCION,TOTAL_CONSULTOR_NOMBRE_PRED,TOTAL_CONSULTOR_NOMBRE_NIVEL_8,PORCENTAJE,PORCENTAJE_PRED
37,Fuerza Comercial Multiservicios Sas,1,9264,11872,22.554048,78.032345
40,INTEGRA. RED DE INTEGRACION S.A.S,1,4573,5478,10.406930,83.479372
28,Conexxion Outsourcing sas,1,3251,5117,9.721114,63.533320
78,Tech Red Sas,1,2328,4167,7.916334,55.867531
72,Servicomunicaciones Caribe B C S A S,1,3285,3470,6.592196,94.668588
82,Ventas 577 Ltda,1,2809,2929,5.564421,95.903039
23,Conecta Telecomunciaciones Sas,1,2850,2896,5.501729,98.411602
46,Inversiones Bitec S A S,1,1808,2442,4.639234,74.037674
59,MyF Soluciones Integrales Occidente,1,1786,2140,4.065504,83.457944
33,Dsurtiendo Sas,1,1318,1897,3.603860,69.478123


In [10]:
dealer_tropas_general.head(20)

,CONSULTOR_NOMBRE_NIVEL_8,PREDICCION,TOTAL_CONSULTOR_NOMBRE_PRED,TOTAL_CONSULTOR_NOMBRE_NIVEL_8,PORCENTAJE,PORCENTAJE_PRED
37,Fuerza Comercial Multiservicios Sas,1,9264,11872,22.554048,78.032345
36,Fuerza Comercial Multiservicios Sas,0,2608,11872,22.554048,21.967655
40,INTEGRA. RED DE INTEGRACION S.A.S,1,4573,5478,10.406930,83.479372
39,INTEGRA. RED DE INTEGRACION S.A.S,0,905,5478,10.406930,16.520628
28,Conexxion Outsourcing sas,1,3251,5117,9.721114,63.533320
27,Conexxion Outsourcing sas,0,1866,5117,9.721114,36.466680
78,Tech Red Sas,1,2328,4167,7.916334,55.867531
77,Tech Red Sas,0,1839,4167,7.916334,44.132469
72,Servicomunicaciones Caribe B C S A S,1,3285,3470,6.592196,94.668588
71,Servicomunicaciones Caribe B C S A S,0,185,3470,6.592196,5.331412


In [100]:
dealer_tropas_general.to_csv('dealers_general_calidad.csv')

In [ ]:
#QUERY PARA regional nombre
query=f'''
SELECT A.*,B.CONSULTOR_REGIONAL_NOMBRE
FROM AGG_DL_PJ_QUALITY A 
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B ON A.CONSULTOR_DK=B.CONSULTOR_DK
WHERE A.PERIODO_PROCESO_CODIGO=202209'''


In [ ]:
#QUERY QUE agrupa por regional
query=f'''
SELECT B.CONSULTOR_REGIONAL_NOMBRE,COUNT(*) AS TOTAL_REGIONAL
FROM AGG_DL_PJ_QUALITY A 
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B ON A.CONSULTOR_DK=B.CONSULTOR_DK
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND A.CONSULTOR_CANAL_NOMBRE='Fuerza De Venta Indirecta'
AND A.CONSULTOR_SUBCANAL_NOMBRE='Tropas'
GROUP BY B.CONSULTOR_REGIONAL_NOMBRE
ORDER BY TOTAL_REGIONAL DESC
'''


In [37]:
#TOTAL DE CADA REGIONAL
subcanal='Tropas'
regional='Regional Centro'
query=f'''
SELECT      A.CONSULTOR_NOMBRE_NIVEL_8,
            COUNT(*) AS TOTAL_CONSULTOR_NOMBRE_NIVEL_8
FROM AGG_DL_PJ_QUALITY A
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B ON A.CONSULTOR_DK=B.CONSULTOR_DK
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND A.CONSULTOR_CANAL_NOMBRE='Fuerza De Venta Indirecta'
AND A.CONSULTOR_SUBCANAL_NOMBRE='{subcanal}'
AND B.CONSULTOR_REGIONAL_NOMBRE='{regional}'
GROUP BY A.CONSULTOR_NOMBRE_NIVEL_8
ORDER BY A.CONSULTOR_NOMBRE_NIVEL_8
'''
dealer_tropas=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [38]:
suma=dealer_tropas['TOTAL_CONSULTOR_NOMBRE_NIVEL_8'].sum()
dealer_tropas['PORCENTAJE']=dealer_tropas['TOTAL_CONSULTOR_NOMBRE_NIVEL_8']/suma *100

In [39]:
#QUERY DE CADA PREDICCION POR REGIONAL
subcanal='Tropas' 
regional='Regional Centro'
query=f'''
SELECT      A.CONSULTOR_NOMBRE_NIVEL_8,
            A.PREDICCION,
            COUNT(*) AS TOTAL_CONSULTOR_NOMBRE_PRED
FROM AGG_DL_PJ_QUALITY A 
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B ON A.CONSULTOR_DK=B.CONSULTOR_DK
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND A.CONSULTOR_CANAL_NOMBRE='Fuerza De Venta Indirecta'
AND A.CONSULTOR_SUBCANAL_NOMBRE='{subcanal}'
AND B.CONSULTOR_REGIONAL_NOMBRE='{regional}'
GROUP BY A.CONSULTOR_NOMBRE_NIVEL_8,A.PREDICCION
ORDER BY A.CONSULTOR_NOMBRE_NIVEL_8,A.PREDICCION
'''
dealer_tropas_pred=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [40]:
#ARREGLOS ESTANDAR 
dealer_tropas_general=dealer_tropas_pred.merge(dealer_tropas,left_on='CONSULTOR_NOMBRE_NIVEL_8',right_on='CONSULTOR_NOMBRE_NIVEL_8',how='left')
dealer_tropas_general['PORCENTAJE_PRED']=dealer_tropas_general['TOTAL_CONSULTOR_NOMBRE_PRED']/dealer_tropas_general['TOTAL_CONSULTOR_NOMBRE_NIVEL_8']*100
dealer_tropas_general=dealer_tropas_general.sort_values(['TOTAL_CONSULTOR_NOMBRE_NIVEL_8','PREDICCION'],ascending=False)

In [41]:
dealer_tropas_general[dealer_tropas_general['PREDICCION']==1]

,CONSULTOR_NOMBRE_NIVEL_8,PREDICCION,TOTAL_CONSULTOR_NOMBRE_PRED,TOTAL_CONSULTOR_NOMBRE_NIVEL_8,PORCENTAJE,PORCENTAJE_PRED
10,Conexxion Outsourcing sas,1,3251,5117,40.877137,63.533320
26,Tech Red Sas,1,1972,3586,28.646749,54.991634
28,Ventas 577 Ltda,1,2809,2929,23.398306,95.903039
5,Comunicamos Colombia Sas,1,310,365,2.915801,84.931507
12,Dsurtiendo Sas,1,55,177,1.413964,31.073446
30,aliado profesional sas,1,70,165,1.318102,42.424242
24,Sin Informacion,1,33,68,0.543218,48.529412
1,Avatar Telcos s.a.s,1,27,48,0.383448,56.250000
20,Organizacion Acceso Colombia,1,1,13,0.103850,7.692308
17,Lion Quick Comunicaciones Sas,1,1,11,0.087873,9.090909


In [35]:
#QUERY QUE DA EL CANAL DE VENTA POR REGION 
query=f'''
SELECT B.CONSULTOR_REGIONAL_NOMBRE,A.PREDICCION,COUNT(*) AS TOTAL_REGIONAL
FROM AGG_DL_PJ_QUALITY A 
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B ON A.CONSULTOR_DK=B.CONSULTOR_DK
WHERE A.PERIODO_PROCESO_CODIGO=202210
AND A.CONSULTOR_CANAL_NOMBRE='Fuerza De Venta Indirecta'
AND A.CONSULTOR_SUBCANAL_NOMBRE='Tropas'
GROUP BY B.CONSULTOR_REGIONAL_NOMBRE,A.PREDICCION
ORDER BY CONSULTOR_REGIONAL_NOMBRE,PREDICCION DESC
'''
regional_pred=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [36]:
regional_pred=regional_pred.merge(dealer_tropas,left_on='CONSULTOR_REGIONAL_NOMBRE',right_on='CONSULTOR_NOMBRE_NIVEL_8',how='left')
dealer_tropas_general['PORCENTAJE_PRED']=dealer_tropas_general['TOTAL_CONSULTOR_NOMBRE_PRED']/dealer_tropas_general['TOTAL_CONSULTOR_NOMBRE_NIVEL_8']*100
dealer_tropas_general=dealer_tropas_general.sort_values(['TOTAL_CONSULTOR_NOMBRE_NIVEL_8','PREDICCION'],ascending=False)